**REPLICATION CODE: MOURIFIÉ/HENRY/MEANGO**

Start date: 10/11/2025
Author: Jordi Torres Vallverdú

Goal: idea is to replicate the analysis done by Mourifié et al (2020) in another context with another data. The idea is to seek new applications of their methodology.

Notes on the context: I am using data on Belgium region of Flandes in the 1990s. I have a panel of students observed from the beginning of high-school up to two years into higher education. Given that I don't observe future labor outcomes, I will only consider potential outcomes in terms of success probability of having completed the first year of higher education (this is a proxy for completion)


$D=\{1,0\}$, where 1 is entered stem high school track and 0 if entered non-stem high-school track

$Y=\{1,0\}$, where 1 is graduated from stem degree and 0 is graduated from non-stem degree

Z, set of instruments


In [99]:
using StatFiles, DataFrames

data_path = "/Users/jorditorresvallverdu/Library/Mobile Documents/com~apple~CloudDocs/tse/year1/papers_final_memoire/data/clean/roymodel_database.dta"

export_dir = "/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/set_identification/"


main = DataFrame(load(data_path))

Row,ID,hs_course1,hs_stem1,postsec_type2,postsec_stem2,c_par_beliefpar_HS_E,c_par_beliefpar_postHS,c_par_childage_homechange,c_par_notsureifB,c_par_yearsinkindergarten,c_par_repeatelementaryeduc,c_par_specialeducelem,c_par_income,c_par_home,c_par_spokenlanguage_dutch_fa,c_par_dialect_fa,c_par_spokenlanguage_dutch_mo,c_par_dialect_mo,c_par_parentschoose,c_par_childchoose,c_par_schoolchoose,c_par_otherchoose,c_par_nochoose,c_bao_advies_school_p0,c_bao_advies_progr_p0,c_bao_advies_welkprogr_p0,c_bao_belief_lkr_endpoint_p0,c_bao_special_p0,c_bao_result_dutch_p0,c_bao_result_math_p0,c_bao_result_all_p0,c_iq_male,c_iq_birth_year,c_iq_birth_month,c_iq_WB1,c_iq_NB1,c_iq_NIQ,c_iq_VIQ,c_iq_SIQ,c_iq_IQ,c_iq_opl,c_sskills_CON1,c_sskills_CON2,c_sskills_AGREE1,c_sskills_AGREE2,c_sskills_EXTRA1,c_sskills_CON3,c_sskills_AGREE3,c_sskills_EXTRA2,c_sskills_EXTRA3,c_sskills_workparents,c_sskills_income,c_sskills_IQ_discrete,z_numeric_iq,income_reduced
,Int32?,Float32?,Float32?,Float32?,Float32?,Int32?,Int32?,Int8?,Int8?,Int8?,Int8?,Int8?,Int32?,Int32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Int8?,Int8?,Int32?,Int32?,Float32?,Int32?,Int32?,Int32?,Int8?,Int16?,Int8?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Float32?,Float32?,Float32?,Int8?,Float32?
1,11135,6.0,1.0,2.0,0.0,6,4,missing,0,3,0,0,4,1,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1,1,7,0.0,5,4,4,1,1978,8,-0.418461,0.609975,91.4869,120.331,109.205,110.171,2,5,5,4,3,4,5,2,4,3,1.0,3.0,5.0,1,2.0
2,11137,6.0,0.0,2.0,0.0,6,4,missing,0,3,0,0,5,1,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0,1,1,1,0.0,5,4,5,0,1978,10,-0.332319,0.560348,97.4658,107.186,100.036,102.72,1,5,5,5,5,5,5,5,5,5,1.0,4.0,4.0,1,3.0
3,11154,6.0,1.0,2.0,0.0,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,1,1,1,7,0.0,5,5,5,1,1978,5,0.57565,1.49024,106.991,115.378,104.474,111.563,2,5,5,5,5,5,5,5,5,5,missing,1.0,5.0,3,1.0
4,11161,6.0,1.0,1.0,1.0,6,4,missing,0,4,0,0,5,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,5,5,missing,0,1978,2,0.221887,1.49837,110.288,115.582,99.4507,111.189,3,5,5,5,5,5,5,5,5,5,1.0,4.0,5.0,3,3.0
5,11162,6.0,1.0,1.0,0.0,6,5,missing,0,3,0,0,4,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,5,5,5,0,1978,2,0.988978,0.905644,118.363,100.334,90.7207,103.559,1,5,5,5,5,5,5,5,5,4,1.0,3.0,4.0,4,2.0
6,11163,6.0,1.0,2.0,0.0,6,4,missing,0,3,0,0,5,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,6,6,missing,0,1978,6,0.959115,0.984085,120.458,118.775,126.786,125.161,3,5,5,5,5,5,5,5,5,2,1.0,4.0,6.0,5,3.0
7,11166,6.0,1.0,2.0,0.0,6,missing,missing,0,3,0,0,missing,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,5,6,missing,0,1978,11,0.158003,1.39593,95.3408,91.9245,89.5506,90.9654,3,5,5,5,5,5,5,5,5,5,missing,1.0,3.0,1,1.0
8,11171,6.0,0.0,1.0,0.0,6,3,missing,0,3,0,0,4,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,6,6,missing,0,1978,5,0.83656,1.99263,110.01,125.206,121.763,123.113,2,5,5,5,5,5,5,5,5,5,1.0,3.0,6.0,3,2.0
9,11172,6.0,1.0,2.0,1.0,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,1,1,1,7,0.0,5,5,5,1,1978,10,1.0285,0.811891,108.808,96.97,99.3044,101.327,3,5,5,5,5,5,5,5,5,4,missing,1.0,4.0,3,1.0


**1. Data manipulation**

First we create a subsample of observations on gender and socio-economic status, just like they do in their exercise. I will create bins based on this: 

- Gender
- Income categories: 6 brackets
- Minority status (proxied by family background)


In [100]:
using DataFramesMeta, Statistics 

groups = groupby(main, [:c_sskills_income, :c_iq_male])
subsamples = [DataFrame(g) for g in groups]


10-element Vector{DataFrame}:
 305×55 DataFrame
 Row │ ID      hs_course1  hs_stem1  postsec_type2  postsec_stem2  c_par_belie ⋯
     │ Int32?  Float32?    Float32?  Float32?       Float32?       Int32?      ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │  11166         6.0       1.0            2.0            0.0              ⋯
   2 │  11246         6.0       0.0            1.0            0.0
   3 │  11272         6.0       1.0            2.0            0.0
   4 │  11311         6.0       1.0            2.0            0.0
   5 │  11457         6.0       0.0            2.0            0.0              ⋯
   6 │  11473         6.0       0.0            2.0            0.0
   7 │  12652         6.0       0.0            2.0            0.0
   8 │  12666         4.0       0.0            2.0            0.0             
  ⋮  │   ⋮         ⋮          ⋮            ⋮              ⋮                 ⋮  ⋱
 299 │  42322         4.0       0.0            2.0       

**2. Test for monothonicity of E(Y/Z=z)**
This boils down to testing whether the probability of Y=1 is increasing in Z. Usual tests include Hu, Shi (2019), Chetverikhov (2013). These tests, however, are done under much more complicated cases as in mine: Hu et al, for example, account for a setting with Y being continuous and Z also being continuous; in my case, I just need to test over Z=1,2,3...

Given that the idea is to use set identification techniques, I will provide some evidence that this holds or not for a given subset of the data. Then I will focus mostly on the el(z) measure. 

In [101]:
size_subsample=10
size_p=3

m= zeros(Float64, size_subsample, size_p, size_p)

10×3×3 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [102]:
##Estimate probabilities 

function cell_probs(df, instrument::Symbol)
    zvals = sort(unique(df[!, instrument]))
    k = length(zvals)
    p_k = zeros(Float64, k)
    n_k = zeros(Int, k)

    for (i, z) in enumerate(zvals)
        num = sum((df.postsec_stem2 .== 1) .&
                  (df[!, instrument] .== z))

        n_k[i] = sum(df[!, instrument] .== z)
        p_k[i] = num / n_k[i]
    end

    return p_k, n_k
end



probs= [cell_probs(i, :z_numeric_iq) for i in subsamples]

10-element Vector{Tuple{Vector{Float64}, Vector{Int64}}}:
 ([0.3076923076923077, 0.2835820895522388, 0.3484848484848485, 0.2631578947368421, 0.5813953488372093], [91, 67, 66, 38, 43])
 ([0.53125, 0.4888888888888889, 0.47058823529411764, 0.640625, 0.5866666666666667], [32, 45, 51, 64, 75])
 ([0.16176470588235295, 0.25, 0.3695652173913043, 0.2571428571428571, 0.4482758620689655], [68, 52, 46, 35, 29])
 ([0.7619047619047619, 0.5185185185185185, 0.42857142857142855, 0.7391304347826086, 0.5172413793103449], [21, 27, 21, 23, 29])
 ([0.28378378378378377, 0.27586206896551724, 0.42424242424242425, 0.2545454545454545, 0.3870967741935484], [74, 58, 66, 55, 31])
 ([0.4375, 0.5428571428571428, 0.7142857142857143, 0.7380952380952381, 0.5777777777777777], [32, 35, 28, 42, 45])
 ([0.1951219512195122, 0.2786885245901639, 0.32558139534883723, 0.32608695652173914, 0.425], [41, 61, 43, 46, 40])
 ([0.21428571428571427, 0.37037037037037035, 0.6153846153846154, 0.5952380952380952, 0.6666666666666666], [14, 2

In [103]:
function compute_var(p_k, n_k)
    k = length(p_k)
    var = zeros(k)
    for j in 1:k
        var[j] = p_k[j] * (1 - p_k[j]) / n_k[j]
    end
    return var
end


compute_var (generic function with 1 method)

In [104]:
#Generate moment inequalities 
function monotonicity_AS(df, instrument::Symbol)

    p_k, n_k = cell_probs(df, instrument)
    var = compute_var(p_k, n_k)

    K = length(p_k)
    if K < 2
        error("Need at least 2 Z categories for monotonicity test.")
    end

    J = K - 1  # number of inequalities
    g = zeros(J)
    sig = zeros(J)

    for j in 1:J
        g[j] = p_k[j+1] - p_k[j]
        sig[j] = sqrt(var[j+1] + var[j])
    end

    n = nrow(df)
    tvals = zeros(J)
    for j in 1:J
        if sig[j] > 0
            tvals[j] = sqrt(n) * (-min(g[j], 0)) / sig[j]
        else
            tvals[j] = 0.0
        end
    end

    T = maximum(tvals)

    return g, sig, T
end



monotonicity_AS (generic function with 1 method)

In [105]:
T_vals = Float64[]
g_list = []
sig_list = []

for df_sub in subsamples
    g, sig, T = monotonicity_AS(df_sub, :z_numeric_iq)
    push!(T_vals, T)
    push!(g_list, g)
    push!(sig_list, sig)
end

T_vals ##same rejections as before


10-element Vector{Float64}:
 16.122605024153746
 10.668163606239695
 16.62087705671087
 20.019122754715095
 33.81824955627914
 21.602359965518342
 -0.0
  2.0919284455219196
  8.826095108049113
 12.304127970399858

In [106]:
using Distributions
using Random

function boot_t_1(subsamples, instrument::Symbol, b_iter::Int64)

    nsubs = length(subsamples)
    T_dist = zeros(nsubs, b_iter)

    for b in 1:b_iter
        for i in 1:nsubs

            dt = subsamples[i]
            n_i = nrow(dt)

            hat_p = mean(dt.postsec_stem2)

            Y_b = rand.(Bernoulli(hat_p), n_i)

            dt_boot = deepcopy(dt)
            dt_boot.postsec_stem2 = Y_b

            _, _, T_dist[i, b] = monotonicity_AS(dt_boot, instrument)

        end
    end

    return T_dist
end


boot_t_1 (generic function with 1 method)

In [107]:
Random.seed!(12345)

T_dist = boot_t_1(subsamples, :z_numeric_iq, 1000)

# critical values
cv_95 = [quantile(T_dist[i,:], 0.95) for i in 1:length(subsamples)]
cv_90 = [quantile(T_dist[i,:], 0.90) for i in 1:length(subsamples)]
cv_85 = [quantile(T_dist[i,:], 0.85) for i in 1:length(subsamples)]


cv_85

10-element Vector{Float64}:
 32.214431607298835
 29.98806202038304
 27.938009747511877
 20.450189607911067
 31.36769631963883
 24.35784764359122
 28.3206902673743
 23.708120212899164
 28.14979772499695
 23.24995959787813

In [112]:
using DataFrames

function monot_table(subsamples, T_vals, cv_95, cv_90, cv_85)

    n = length(subsamples)

    df_out = DataFrame(
        subsample = Int[],
        income = String[],
        gender = String[],
        T_stat = Float64[],
        cv_95 = Float64[],
        cv_90 = Float64[],
        cv_85 = Float64[],
        reject_95 = Int[],
        reject_90 = Int[],
        reject_85 = Int[]
    )

    for i in 1:n

        income_i = string(unique(subsamples[i].c_sskills_income)[1])
        gender_i = string(unique(subsamples[i].c_iq_male)[1])

        T_i = T_vals[i]

        r95 = T_i >= cv_95[i] ? 1 : 0
        r90 = T_i >= cv_90[i] ? 1 : 0
        r85 = T_i >= cv_85[i] ? 1 : 0

        push!(df_out, (
            i,
            income_i,
            gender_i,
            T_i,
            cv_95[i],
            cv_90[i],
            cv_85[i],
            r95,
            r90,
            r85
        ))
    end

    return df_out
end

df = monot_table(subsamples, T_vals, cv_95, cv_90, cv_85)


Row,subsample,income,gender,T_stat,cv_95,cv_90,cv_85,reject_95,reject_90,reject_85
,Int64,String,String,Float64,Float64,Float64,Float64,Int64,Int64,Int64
1,1,1.0,0,16.1226,40.6511,35.3571,32.2144,0,0,0
2,2,1.0,1,10.6682,38.7424,32.9932,29.9881,0,0,0
3,3,2.0,0,16.6209,36.4752,31.2896,27.938,0,0,0
4,4,2.0,1,20.0191,26.1864,22.583,20.4502,0,0,0
5,5,3.0,0,33.8182,40.1014,34.8096,31.3677,0,0,1
6,6,3.0,1,21.6024,32.1399,27.3157,24.3578,0,0,0
7,7,4.0,0,-0.0,35.6309,31.7686,28.3207,0,0,0
8,8,4.0,1,2.09193,30.8177,27.0984,23.7081,0,0,0
9,9,5.0,0,8.8261,35.203,31.043,28.1498,0,0,0


**USING PAV**

In [59]:
#Estimate test

function pav3(pk, nk)
    p1, p2, p3 = pk
    n1, n2, n3 = nk

  #monothonicity holds
  if p1 ≤ p2 ≤ p3
    return pk
  end


  #case 1
  if p1>=p2
    
    θ_12= (p1*n1 + p2*n2)/(n1+n2)

    if p3>=θ_12
      return[θ_12, θ_12, p3]
    else 
    θ_123= (p1*n1 + p2*n2 + p3*n3)/(n1+n2+n3)

    return[θ_123,θ_123,θ_123]
    end 
  end 
  
  #case 2
  if p2>=p3

    θ_23= (p2*n2 + p3*n3)/(n2+n3)

    if p1<θ_23
      return[p1, θ_23, θ_23]
    else 
      θ_123= (p1*n1 + p2*n2 + p3*n3)/(n1+n2+n3)
     return[θ_123,θ_123,θ_123]
    end 

  end 


end


pav3 (generic function with 1 method)

In [ ]:
##extend the function to apply to any number of pair comparisons. 

In [58]:
#To check it works: 

pk = [0.40, 0.20, 0.50]
nk = [50, 60, 40]

pav3(pk, nk)


3-element Vector{Float64}:
 0.2909090909090909
 0.2909090909090909
 0.5

In [60]:
p_hat= zeros(length(subsamples), length(zvals))

for i in 1:length(subsamples)

    pk, nk = probs[i]

    p_hat[i,:]= pav3(pk,nk)
end 

p_hat

10×3 Matrix{Float64}:
 0.302158  0.333333  0.491803
 0.5       0.53012   0.6
 0.194175  0.297297  0.415094
 0.61157   0.61157   0.61157
 0.254386  0.364706  0.364706
 0.462963  0.648438  0.648438
 0.251534  0.251534  0.382353
 0.305556  0.612903  0.612903
 0.207792  0.360825  0.478873
 0.484848  0.58      0.616438

In [61]:
n=zeros(length(subsamples))
T=zeros(length(subsamples))
[n[i]= nrow(subsamples[i]) for i in 1:length(subsamples)]

for i in 1:length(subsamples)

    pk, nk= probs[i]
    p_hat[i,:]= pav3(pk,nk)

    T[i]= sqrt(n[i])* maximum(pk .- p_hat[i,:])
end

T

10-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.24833702882483422
 0.08921805642186757
 0.1656228941914646
 0.07411659725667598
 0.4839494693731058
 0.0
 0.0

In [62]:
for i in 1:length(subsamples)
    println((i, unique(subsamples[i].c_sskills_income), unique(subsamples[i].c_iq_male), T[i]))
end


(1, Union{Missing, Float32}[1.0], Union{Missing, Int8}[0], 0.0)
(2, Union{Missing, Float32}[1.0], Union{Missing, Int8}[1], 0.0)
(3, Union{Missing, Float32}[2.0], Union{Missing, Int8}[0], 0.0)
(4, Union{Missing, Float32}[2.0], Union{Missing, Int8}[1], 0.24833702882483422)
(5, Union{Missing, Float32}[3.0], Union{Missing, Int8}[0], 0.08921805642186757)
(6, Union{Missing, Float32}[3.0], Union{Missing, Int8}[1], 0.1656228941914646)
(7, Union{Missing, Float32}[4.0], Union{Missing, Int8}[0], 0.07411659725667598)
(8, Union{Missing, Float32}[4.0], Union{Missing, Int8}[1], 0.4839494693731058)
(9, Union{Missing, Float32}[5.0], Union{Missing, Int8}[0], 0.0)
(10, Union{Missing, Float32}[5.0], Union{Missing, Int8}[1], 0.0)


In [63]:
using Distributions
using Random
Random.seed!(12345)   #seed



T_dist= zeros(length(subsamples), b_iter)

##set seed!!!
function boot_t(instrument::Symbol, b_iter::Int64)

 T_dist= zeros(length(subsamples), b_iter)

  for b in 1:b_iter
    for i in 1:length(subsamples)

          dt = subsamples[i]   ##then we loop over i 
          p_iso = p_hat[i,:]   
          Z = dt[!, instrument] 
          p_obs = p_iso[Z]   
          Y_b = rand.(Bernoulli.(p_obs))

          dt_boot = deepcopy(dt)       
          dt_boot.postsec_stem2 = Y_b   #simul replace

          pk_b, nk_b= cell_probs(dt_boot)
          p_iso_b= pav3(pk_b,nk_b)
          T_dist[i,b]= sqrt(n[i])* maximum(pk_b .- p_iso_b)
    end 
  end
  
  return T_dist

end 


T_dist= boot_t(:z_numeric_iq, 1000)





10×1000 Matrix{Float64}:
 0.37606   1.02213   0.645413   0.0712522  …  0.0        0.159903   0.700225
 0.100865  0.686789  0.803334   1.00487       0.100865   0.140115   0.62559
 0.0       0.285732  0.158169   1.55785       0.219793   0.267505   0.197814
 0.476077  0.964912  0.72488    0.724432      0.0964912  0.0572917  0.212316
 0.0       0.695492  0.0946218  0.409921      0.214904   0.333582   0.0478241
 0.271812  0.721133  0.430831   0.0        …  0.772907   0.0        0.0
 0.0       1.63391   2.80726    1.92999       0.522191   2.77436    0.844371
 1.63654   0.0       0.0        1.97749       4.15955    2.23618    0.0143781
 2.26801   2.60875   2.843      0.434791      0.628229   0.0        0.434791
 0.0       4.40352   0.0        0.367353      0.367353   0.0        2.75515

In [64]:
using Statistics

c_value_95= zeros(length(subsamples))
c_value_90= zeros(length(subsamples))

[c_value_95[i]= quantile(T_dist[i, :], 0.95) for i in 1:length(subsamples)]
[c_value_90[i]= quantile(T_dist[i, :], 0.90) for i in 1:length(subsamples)]



10-element Vector{Float64}:
 0.8568914345243583
 1.2126469734397252
 0.7692772189617516
 0.7719298245614035
 1.169145451432091
 0.9319259500916867
 1.9628892232966446
 2.829854941174938
 4.621207153499565
 4.224557469504799

In [65]:
t_result95= zeros(length(subsamples))
t_result90= zeros(length(subsamples))

for i in 1:length(subsamples)
    if (T[i]>=c_value_95[i])
        t_result95[i]=1
    else 
        t_result95[i]=0
    end

    if (T[i]>=c_value_90[i])
        t_result90[i]=1
    else 
        t_result90[i]=0
    end


end 

tests= [t_result95 t_result90]

10×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

**Using IQ as instrument**-->classic Manski Pepper type of instrument






**2. Estimating bounds on the L(z) measure**

The goal is now to understand what explains the violations of Roy selection model using the same method as in Mourifié, Henry, Meango (2020). Will keep the same example going, but, if needed, I will also include a continuous instrument: example can be numeric IQ estimated at baseline. 

The main issue with this literature is that sample analogs of $\Theta_I$ are the suprema and infima of estimated bounding functions and they have substantial finite sample bias and estimated bounds tend to be much tighter than population bounds. -->Manski/Pepper (2001).

I think I need to treat the instrument as continuous; if not it is meaningless. Because CRL seems to be devised with this property. Then I probably need to adapt the test for monothonicity too. But so far, I have something, which is fine.

Start simple; support is discrete; then we do the same exercise with a continuous instrument. 

In [13]:
##Estimate probabilities 

function cell_probs_0(df, instrument::Symbol)

    zvals = sort(unique(df[!, instrument]))  
    k = length(zvals)

    p_k = Dict{Int, Float64}()   
    n_k = Dict{Int, Int}()

    for z in zvals
        sub = df[df[!, instrument] .== z, :]

        n = nrow(sub)
        n_k[z] = n

        if n == 0
            p_k[z] = NaN
        else
            p_k[z] = sum(sub.postsec_stem2 .== 0) / n
        end
    end

    return zvals, p_k, n_k
end


probs= [cell_probs_0(i, :c_iq_opl) for i in subsamples]



10-element Vector{Tuple{Vector{Union{Missing, Int8}}, Dict{Int64, Float64}, Dict{Int64, Int64}}}:
 ([1, 2, 3], Dict(2 => 0.6296296296296297, 3 => 0.5444444444444444, 1 => 0.7454545454545455), Dict(2 => 108, 3 => 90, 1 => 110))
 ([1, 2, 3], Dict(2 => 0.3924050632911392, 3 => 0.4351851851851852, 1 => 0.45121951219512196), Dict(2 => 79, 3 => 108, 1 => 82))
 ([1, 2, 3], Dict(2 => 0.6956521739130435, 3 => 0.6410256410256411, 1 => 0.7903225806451613), Dict(2 => 69, 3 => 39, 1 => 124))
 ([1, 2, 3], Dict(2 => 0.47368421052631576, 3 => 0.3157894736842105, 1 => 0.42424242424242425), Dict(2 => 38, 3 => 19, 1 => 66))
 ([1, 2, 3], Dict(2 => 0.6190476190476191, 3 => 0.686046511627907, 1 => 0.72), Dict(2 => 126, 3 => 86, 1 => 75))
 ([1, 2, 3], Dict(2 => 0.4507042253521127, 3 => 0.2909090909090909, 1 => 0.40350877192982454), Dict(2 => 71, 3 => 55, 1 => 57))
 ([1, 2, 3], Dict(2 => 0.6666666666666666, 3 => 0.6778523489932886, 1 => 0.8333333333333334), Dict(2 => 69, 3 => 149, 1 => 18))
 ([1, 2, 3], Dict(

In [14]:
function compute_el(df, instrument::Symbol)

    zvals, p_k, n_k = cell_probs_0(df, instrument)

    el = zeros(length(zvals))

    running_min = Inf

    for (i, z) in enumerate(zvals)
        running_min = min(running_min, p_k[z])
        el[i] = p_k[z] - running_min
    end

    return zvals, p_k, n_k, el
end


zvals, p_k, n_k, el = compute_el(subsamples[10], :c_iq_opl)


(Union{Missing, Int8}[1, 2, 3], Dict(2 => 0.6, 3 => 0.4357142857142857, 1 => 0.25), Dict(2 => 15, 3 => 140, 1 => 4), [0.0, 0.35, 0.18571428571428572])

In [15]:
#Generate moment inequalities

function m_ineq(theta, p_k, n_k, zvals_s)
    n = length(zvals_s)
    moment = zeros(n,n)
    se_moment = zeros(n,n)

    for ii in 1:n
        z_i = zvals_s[ii]

        for jj in 1:ii
            z_j = zvals_s[jj]

            if n_k[z_i] == 0 || n_k[z_j] == 0
                moment[ii,jj] = 0.0
                se_moment[ii,jj] = Inf
                continue
            end

            moment[ii,jj] = (p_k[z_i] - p_k[z_j]) - theta[ii] ##key point
            var_i = p_k[z_i]*(1 - p_k[z_i]) / n_k[z_i]
            var_j = p_k[z_j]*(1 - p_k[z_j]) / n_k[z_j]

            se_moment[ii,jj] = sqrt(var_i + var_j)
        end
    end

    return moment, se_moment
end



m_ineq (generic function with 1 method)

In [16]:
#put the moment inequalities in a vector

function vectorize(moment::AbstractMatrix)
 n = size(moment, 1)

 K = n * (n+ 1) ÷ 2
 moment_vector= zeros(K)
    k=1
        for j in 1:n
            for y in 1:j
                    moment_vector[k]=moment[j,y]
                    k +=1
            end 
            
        end
     
  return moment_vector

end 

vectorize (generic function with 1 method)

In [17]:
##Now generate T value of the moment for a given value of the grid. 

function T_test(df, instrument, theta)

    zvals, p_k, n_k, el= compute_el(df, instrument)

        
    moment, se_moment=m_ineq(theta, p_k, n_k, zvals)

    moment_v= vectorize(moment)
    moment_se= vectorize(se_moment)

    T_value= maximum(moment_v./max.(moment_se, 1e-8))
         
    return T_value

end 

T_test (generic function with 1 method)

In [18]:

function bootstrap(df, moment_not,se_moment_not,theta_not, B, instrument::Symbol, zvals_original)

    T_b= zeros(B)
    for b in 1:B

        df_b = nothing
        while true
            n = nrow(df)
            df_try = df[rand(1:n, n), :]
            zvals_b = sort(unique(df_try[!, instrument]))
            if zvals_b == zvals_original
                df_b = df_try
                break
            end
        end

        zvals, p_k_b, n_k_b, el_b= compute_el(df_b, instrument)
        
        moment, se_moment=m_ineq(theta_not, p_k_b, n_k_b, zvals)

        moment_v_b= vectorize(moment)

        Z_vec_b= @. (moment_v_b - moment_not) /se_moment_not
        T_b[b]=maximum(Z_vec_b)

    end 
    return T_b

end 

bootstrap (generic function with 1 method)

In [19]:
#let's take only the case where it is violated, the main condition...



zvals, p_k, n_k, el = compute_el(subsamples[10], :c_iq_opl)

theta_not=el 
moment_not, se_moment_not= m_ineq(theta_not, p_k, n_k, zvals)

se_not_vec= vectorize(se_moment_not)
moment_not_vec=vectorize(moment_not)


zvals_original= sort(unique(subsamples[10][!, :c_iq_opl]))



3-element Vector{Union{Missing, Int8}}:
 1
 2
 3

In [20]:
T_boot= bootstrap(subsamples[10], moment_not_vec,se_not_vec,theta_not, 1000, :c_iq_opl, zvals_original)


K_alpha= quantile(T_boot, 0.95)



1.9786881975072133

In [21]:
##Apply the code for each subsample and for different instruments. 


function apply_bootsubsamples(instrument::Symbol, B)

    size_s= length(subsamples)
    T_boot_s=zeros(size_s, B)
    
    for s in 1:size_s
        
        zvals, p_k, n_k, el = compute_el(subsamples[s], instrument)

        theta_not=el #should I change this?
        moment_not, se_moment_not= m_ineq(theta_not, p_k, n_k, zvals)

        se_not_vec= vectorize(se_moment_not)
        moment_not_vec=vectorize(moment_not)


        zvals_original= sort(unique(subsamples[s][!, instrument]))

        T_boot_s[s, :]= bootstrap(subsamples[s], moment_not_vec,se_not_vec,theta_not, B, instrument, zvals_original)

        

    end 

    return T_boot_s

end

apply_bootsubsamples (generic function with 1 method)

In [22]:
T_boot_s= apply_bootsubsamples(:c_iq_opl, 1000)


iz= length(subsamples)

alpha_by_subsample= zeros(iz)

[alpha_by_subsample[i]= quantile(T_boot_s[i,:], 0.95) for i in 1:iz]





10-element Vector{Float64}:
 2.1601960170312497
 2.0738836126693445
 2.0564537092948756
 2.1579000526168595
 2.0832948842084993
 2.1478974921286333
 2.106672011149204
 2.2103463347416534
 2.3278194005813493
 1.8118805392327646

In [23]:
##Finally set up the grid, compute T and then compare with critical values simulated via bootstrap. 

#set a grid of values for el(z) do I need to loop over these or jus tby computing the upper bound is enough... ? is it really needed more?

In [24]:
function T_test(df, instrument, theta)

    zvals, p_k, n_k, el= compute_el(df, instrument)

        
    moment, se_moment=m_ineq(theta, p_k, n_k, zvals)

    moment_v= vectorize(moment)
    moment_se= vectorize(se_moment)

    T_value= maximum(moment_v./max.(moment_se, 1e-8))
         
    return T_value

end 

T_test (generic function with 1 method)

In [25]:
T_test(subsamples[10], :c_iq_opl, theta_not)

theta_not

3-element Vector{Float64}:
 0.0
 0.35
 0.18571428571428572

In [26]:
#compute te

function T_simul(subsamples, θ_grid, instrument::Symbol)

    iz = length(subsamples)              
    grid_size = length(θ_grid)           

    T_grid = zeros(iz, grid_size)

    for z in 1:iz
        df_s = subsamples[z]
        for theta_idx in 1:grid_size
            θ = θ_grid[theta_idx]
            T_grid[z, theta_idx] = T_test(df_s, instrument, θ)
        end
    end

    return T_grid
end


T_simul (generic function with 1 method)

In [ ]:

grid1 = [0.0]                     # quite ad hoc coded; needs to in
grid2 = collect(0.0:0.005:0.35)   
grid3 = collect(0.0:0.005:0.185)   


θ_grid = [ [x,y,z] for (x,y,z) in Iterators.product(grid1, grid2, grid3) ]

T_grid= T_simul(subsamples, θ_grid, :c_iq_opl)






10×2698 Matrix{Float64}:
 0.0       0.0       0.0       0.0       …  0.0        0.0        0.0
 0.587967  0.587967  0.587967  0.587967     0.0        0.0        0.0
 0.0       0.0       0.0       0.0          0.0        0.0        0.0
 0.488074  0.438716  0.389357  0.339999     0.0        0.0        0.0
 1.0128    1.0128    1.0128    1.0128       0.0        0.0        0.0
 0.53751   0.480565  0.42362   0.366675  …  0.0        0.0        0.0
 0.1634    0.1634    0.1634    0.1634       0.0        0.0        0.0
 0.139094  0.114212  0.108642  0.108642     0.0        0.0        0.0
 0.0       0.0       0.0       0.0          0.0        0.0        0.0
 1.39582   1.37588   1.35594   1.336        0.0398805  0.0199403  0.00323903

In [57]:
function accepted_theta_per_subsample(T_grid, alpha_by_subsample, θ_grid)

    ns = size(T_grid, 1)      
    ng = size(T_grid, 2)       

    accepted = Vector{Vector{Vector{Float64}}}(undef, ns)

    for s in 1:ns
        k_s = alpha_by_subsample[s]

        θ_accepted = Vector{Vector{Float64}}()

        for g in 1:ng
            if T_grid[s, g] <= k_s
                push!(θ_accepted, θ_grid[g])
            end
        end

        accepted[s] = θ_accepted
    end

    return accepted
end


accepted_theta_per_subsample (generic function with 1 method)

In [58]:
T_grid = T_simul(subsamples, θ_grid, :c_iq_opl)

accepted = accepted_theta_per_subsample(T_grid, alpha_by_subsample, θ_grid)


10-element Vector{Vector{Vector{Float64}}}:
 [[0.0, 0.0, 0.0], [0.0, 0.005, 0.0], [0.0, 0.01, 0.0], [0.0, 0.015, 0.0], [0.0, 0.02, 0.0], [0.0, 0.025, 0.0], [0.0, 0.03, 0.0], [0.0, 0.035, 0.0], [0.0, 0.04, 0.0], [0.0, 0.045, 0.0]  …  [0.0, 0.305, 0.185], [0.0, 0.31, 0.185], [0.0, 0.315, 0.185], [0.0, 0.32, 0.185], [0.0, 0.325, 0.185], [0.0, 0.33, 0.185], [0.0, 0.335, 0.185], [0.0, 0.34, 0.185], [0.0, 0.345, 0.185], [0.0, 0.35, 0.185]]
 [[0.0, 0.0, 0.0], [0.0, 0.005, 0.0], [0.0, 0.01, 0.0], [0.0, 0.015, 0.0], [0.0, 0.02, 0.0], [0.0, 0.025, 0.0], [0.0, 0.03, 0.0], [0.0, 0.035, 0.0], [0.0, 0.04, 0.0], [0.0, 0.045, 0.0]  …  [0.0, 0.305, 0.185], [0.0, 0.31, 0.185], [0.0, 0.315, 0.185], [0.0, 0.32, 0.185], [0.0, 0.325, 0.185], [0.0, 0.33, 0.185], [0.0, 0.335, 0.185], [0.0, 0.34, 0.185], [0.0, 0.345, 0.185], [0.0, 0.35, 0.185]]
 [[0.0, 0.0, 0.0], [0.0, 0.005, 0.0], [0.0, 0.01, 0.0], [0.0, 0.015, 0.0], [0.0, 0.02, 0.0], [0.0, 0.025, 0.0], [0.0, 0.03, 0.0], [0.0, 0.035, 0.0], [0.0, 0.04, 0.0], [

**Extension to account for Z being continuous**--> this should go more in line with Chernozhukov Lee Rosen. 